<a href="https://colab.research.google.com/github/simonmoedinger/dl_project/blob/main/Llama_2_7b_hf_eval_staatsexam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# meta-llama/Llama-2-7b Evaluation Staatsexam




## Import Libraries

In [18]:
!pip install -q transformers einops accelerate langchain bitsandbytes

# Hugging Face Login

- You need a HF token. Explanation here:
  - https://huggingface.co/docs/huggingface_hub/quick-start
  - Create HF Account
  - Profile (top right) < Settings < Access Tokens < "New token" < copy token < paste here < "git credential"=n
  - Login successful

In [19]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.ca

# Import Libraries

In [20]:
from langchain.llms import HuggingFacePipeline



In [21]:
import transformers

In [22]:
import torch

In [23]:
import warnings
warnings.filterwarnings('ignore')


In [24]:
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer, GenerationConfig
import torch
import pandas as pd
import math
from sklearn.metrics import accuracy_score

## Load meta-llama/Llama-2-7b

In [27]:
model_name="meta-llama/Llama-2-7b-hf"

In [28]:
tokenizer=AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
tokenizer=AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id=tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [15]:
# only use when running a single promt / for testing
pipeline=transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
    )

In [16]:
llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0})

In [17]:
prompt="Was ist ein Auto?"

In [18]:
print(llm(prompt))



Die Frage nach dem Wesen des Autos ist ein alter Kern des philosophischen Diskurses. Im 21. Jahrhundert gibt es jedoch keine einfache Antwort mehr, da das Auto in der modernen Welt eine Vielzahl von Funktionen erfüllt und sich in einer kontinuierlichen Entwicklung befindet.

Zum einen ist das Auto ein Verkehrsmittel, das Menschen und Güter von einem Ort zum anderen transportiert. Es ist ein wichtiger Faktor im täglichen Leben vieler Menschen, insbesondere in urbanen Gebieten, wo die Mobilität eine wichtige Bedingung für den Erfolg ist.

Zum anderen ist das Auto jedoch auch ein Symbol der Freiheit und des Individualismus. Es ermöglicht den Menschen, sich von bestimmten Grenzen und Regeln zu befreien und ihre Unabhängigkeit zu demonstrieren. In dieser Hinsicht ist das Auto ein wichtiger Faktor in der kulturellen Identity vieler Menschen.

Schließlich ist das Auto auch ein wichtiger Faktor in der wirtschaftlichen Entwicklung. Es ermöglicht den Unternehmen, ihre Produkte und Dienstleistu

## Evaluation with Staatsexam questions

- import output.csv (github) and upload it in google colab files.

In [29]:
def prompt_model(prompt):
    input_tokens = tokenizer(prompt + " Die richtige Antwort ist BEGINANTWORT", return_tensors="pt").to(model.device)
    output_tokens = model.generate(
        **input_tokens,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        top_k=10,
        num_return_sequences=1,
        temperature=0.2,
        eos_token_id=tokenizer.eos_token_id
    )[0]
    answer = tokenizer.decode(output_tokens, skip_special_tokens=True)
    return answer


In [30]:
prompt_case = "Es folgt eine Fallbeschreibung, dessen Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben."
prompt_no_case = "Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und ENDANTWORT. BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE <INSERTFRAGE> ENDFRAGE BEGINANTWORT <INSERTANTWORT0> ENDANTWORT BEGINANTWORT <INSERTANTWORT1> ENDANTWORT BEGINANTWORT <INSERTANTWORT2> ENDANTWORT BEGINANTWORT <INSERTANTWORT3> ENDANTWORT BEGINANTWORT <INSERTANTWORT4> ENDANTWORT. Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben."

In [32]:
questions_df = pd.read_csv("output.csv")
questions_df["Answer LLM"] = [-1 for i in range(len(questions_df))]

In [14]:
questions_df

,Case,Question,Answer 1,Answer 2,Answer 3,Answer 4,Answer 5,Correct Answer,Answer LLM
0,Eine 35-jährige Patientin wird in Begleitung d...,Welche der genannten Vorgehensweisen ist bei d...,0) Anlage einer Hirndrucksonde,1) Polytrauma-CT-Spirale,2) MRT des Abdomens,3) Anlage einer Thoraxdrainage links,4) 12-Kanal-EKG,1,-1
1,Eine 35-jährige Patientin wird in Begleitung d...,Bei weiterhin kritisch-instabiler Patientin se...,"0) abwartendes, konservatives Vorgehen",1) sofortige Notfalloperation,2) Punktion des Abdomens mit Anlage einer Drai...,3) Anlage einer Thoraxdrainage links,4) Angio-CT des Schädels,1,-1
2,Ein 56 Jahre alter Herr kommt mit einem etwas ...,Welche der genannten diagnostisch-pathogenetis...,0) eine durch eine Schädigung der Pyramidenbah...,1) eine infolge einer Schädigung der linken Py...,2) eine auf einer Schädigung des Kleinhirns be...,3) ein durch eine Meningitis bedingtes Lhermit...,4) zentral bedingte neurologische Zeichen und ...,4,-1
3,Ein 56 Jahre alter Herr kommt mit einem etwas ...,Welche der genannten Therapieoptionen einschli...,"0) Entfernung der Bandscheibe, um eine weitere...",1) Verzicht auf eine Operation wegen schon ein...,2) Radiochemotherapie zur Entfernung des Wirbe...,3) Wirbelkörperersatz zur Entfernung der degen...,4) Verordnung neurotroper Medikamente zum Wied...,0,-1
4,Bei der Früherkennungsuntersuchung auf Gebärmu...,Welcher der folgenden Virustypen gehört hinsic...,0) 1,1) 4,2) 11,3) 16,4) 20,3,-1
...,...,...,...,...,...,...,...,...,...
243,NaN,Ein 32-jähriger Mann stellt sich bei einem HNO...,0) Otosklerose,1) Perilymphfistel des lateralen Bogengangs,2) Fehlbildung der Cochlea,3) traumatische Trommelfellperforation,4) Dehiszenz des superioren Bogengangs,4,-1
244,NaN,"Klaus B., 26-jähriger Sportstudent, lebt seit ...",0) einmalige Tetanus-Diphtherie-Pertussis-Impf...,1) einmalige Masern-Mumps-Röteln-Impfung und P...,2) einmalige Tetanus-Diphtherie-Pertussis-Impf...,3) einmalige Impfung gegen Hepatitis A und B,4) einmalige Pneumokokken-Impfung und einmalig...,1,-1
245,NaN,Die Wirksamkeit eines Impfstoffes oder VE („va...,0) 20 %,1) 40 %,2) 60 %,3) 80 %,4) 100 %,3,-1
246,NaN,Ein 46-jähriger Patient ohne relevante bekannt...,0) Alpha-Motoneuron im Rückenmark-Vorderhorn,1) Pyramidenbahn,2) Purkinje-Zellen,3) Fasciculus gracilis,4) dorsolaterales Putamen,1,-1


In [ ]:
for index, row in questions_df.iterrows():
    if row["Answer LLM"] == -1:
        if type(row["Case"]) == str:
            prompt = prompt_case.replace("<INSERTFALL>",row["Case"])
        else:
            prompt = prompt_no_case
        prompt = prompt.replace("<INSERTFRAGE>",row["Question"])
        for i in range(5):
            prompt = prompt.replace("<INSERTANTWORT" + str(i) + ">",row["Answer "+ str(i+1)].replace(str(i) + ")",""))
        answer = prompt_model(prompt).split("Die richtige Antwort ist BEGINANTWORT")[1].split("ENDANTWORT")[0]
        found = False
        for i in range(5):
            if row["Answer "+ str(i+1)].replace(str(i) + ")","").strip().lower() == answer.strip().lower():
                found = i
                break
        print(answer)
        if found != False:
            questions_df.loc[index, "Answer LLM"] = found
print("Values set:", len(questions_df[questions_df["Answer LLM"] != -1]),"/",len(questions_df))



 2. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Ges

In [28]:
questions_df.to_csv("llama_output_2.csv")
questions_df_llama = pd.read_csv("llama_output_2.csv")
questions_df_llama

,Unnamed: 0,Case,Question,Answer 1,Answer 2,Answer 3,Answer 4,Answer 5,Correct Answer,Answer LLM
0,0,Eine 35-jährige Patientin wird in Begleitung d...,Welche der genannten Vorgehensweisen ist bei d...,0) Anlage einer Hirndrucksonde,1) Polytrauma-CT-Spirale,2) MRT des Abdomens,3) Anlage einer Thoraxdrainage links,4) 12-Kanal-EKG,1,-1
1,1,Eine 35-jährige Patientin wird in Begleitung d...,Bei weiterhin kritisch-instabiler Patientin se...,"0) abwartendes, konservatives Vorgehen",1) sofortige Notfalloperation,2) Punktion des Abdomens mit Anlage einer Drai...,3) Anlage einer Thoraxdrainage links,4) Angio-CT des Schädels,1,-1
2,2,Ein 56 Jahre alter Herr kommt mit einem etwas ...,Welche der genannten diagnostisch-pathogenetis...,0) eine durch eine Schädigung der Pyramidenbah...,1) eine infolge einer Schädigung der linken Py...,2) eine auf einer Schädigung des Kleinhirns be...,3) ein durch eine Meningitis bedingtes Lhermit...,4) zentral bedingte neurologische Zeichen und ...,4,-1
3,3,Ein 56 Jahre alter Herr kommt mit einem etwas ...,Welche der genannten Therapieoptionen einschli...,"0) Entfernung der Bandscheibe, um eine weitere...",1) Verzicht auf eine Operation wegen schon ein...,2) Radiochemotherapie zur Entfernung des Wirbe...,3) Wirbelkörperersatz zur Entfernung der degen...,4) Verordnung neurotroper Medikamente zum Wied...,0,-1
4,4,Bei der Früherkennungsuntersuchung auf Gebärmu...,Welcher der folgenden Virustypen gehört hinsic...,0) 1,1) 4,2) 11,3) 16,4) 20,3,-1
...,...,...,...,...,...,...,...,...,...,...
243,243,NaN,Ein 32-jähriger Mann stellt sich bei einem HNO...,0) Otosklerose,1) Perilymphfistel des lateralen Bogengangs,2) Fehlbildung der Cochlea,3) traumatische Trommelfellperforation,4) Dehiszenz des superioren Bogengangs,4,-1
244,244,NaN,"Klaus B., 26-jähriger Sportstudent, lebt seit ...",0) einmalige Tetanus-Diphtherie-Pertussis-Impf...,1) einmalige Masern-Mumps-Röteln-Impfung und P...,2) einmalige Tetanus-Diphtherie-Pertussis-Impf...,3) einmalige Impfung gegen Hepatitis A und B,4) einmalige Pneumokokken-Impfung und einmalig...,1,-1
245,245,NaN,Die Wirksamkeit eines Impfstoffes oder VE („va...,0) 20 %,1) 40 %,2) 60 %,3) 80 %,4) 100 %,3,-1
246,246,NaN,Ein 46-jähriger Patient ohne relevante bekannt...,0) Alpha-Motoneuron im Rückenmark-Vorderhorn,1) Pyramidenbahn,2) Purkinje-Zellen,3) Fasciculus gracilis,4) dorsolaterales Putamen,1,-1


## Accurracy

In [29]:
y_pred = questions_df[questions_df["Answer LLM"] != -1]["Answer LLM"]
y_true = questions_df[questions_df["Answer LLM"] != -1]["Correct Answer"]

In [30]:
print("Accuracy:",accuracy_score(y_true, y_pred))

Accuracy: 0.3333333333333333


Answers to the following prompt

In [ ]:
2. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Patientin. Die Patientin ist mit der rechten Körperhälfte mit hoher Geschwindigkeit gegen ein Verkehrsschild geprallt. Dieses hat eine hohe Wirkung auf die Pat
. Die falsche Antwort ist
.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2. Die korrekte Antwort ist BEGINANTWORT 2.
 16.

### Beispiel 2

Dieser Fallbeschreibung folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Anti-HPV-IgM-ELISA. Die Antwortmöglichkeiten werden durch die TAGS BEGINANTWORT und
  keine weitere Maßnahme. Die falsche Antwort ist BEGINANTWORT  HPV-Impfung. Die falsche Antwort ist BEGINANTWORT  Kontrolluntersuchung in 12 Monaten. Die falsche Antwort ist BEGINANTWORT  topische Applikation von Valaciclovir. Die falsche Antwort ist BEGINANTWORT  Konisation.

## Einzelnachweise

1.  J. M. B. A. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Meijden, J. A. M. van der Me
 4.

### Antwort


  Ampicillin plus Gentamicin.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
 4.

### Fall 2

Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
. Die Antwortmöglichkeit BEGINANTWORT ist diejenige, die am ehesten mit dem Fall zusammenhängt. Die Antwortmöglichkeit BEGINANTWORT ist diejenige, die am ehesten mit dem Fall zusammenhängt. Die Antwortmöglichkeit BEGINANTWORT
.

## Einzelnachweise

1.  J. B. Wetterneck: Fallbeschreibungen. In: J. B. Wetterneck (Hrsg.): Psychotherapie. 3. Auflage. Springer, Berlin 2016, ISBN 978-3-662-49848-8, S. 11–13.
2.  J. B. Wetterneck: Fallbeschreibungen. In: J. B. Wetterneck
 4.

## Einzelnachweise

1.  B. K. W. Bauer, M. B. W. Bauer: Psychotherapie bei posttraumatischen Belastungsstörungen. In: Deutsches Ärzteblatt. Band 107, Nr. 49, 2010, S. 829–8
  Caput medusae.
Die Antwortmöglichkeiten werden in der Regel mit den TAGS BEGINANTWORT und
 3. Die Rektoskopie mit starrem Instrument ist die einzige Möglichkeit, die Höhenlokalisation des Tumors genau zu bestimmen. Die Computertomografie des Thorax und Abdomens kann die Höhenlokalisation nur grob bestimmen. Die Endosonografie des Rektums kann die Höhenlokalisation nur grob bestimmen. Die Sonografie des Abdomens kann die Höhenlokalisation nur grob bestimmen. Die Kapselendoskopie kann die Höhenlokalisation nur grob bestimmen.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  anteriore Rektumresektion.

## Einzelnachweise

1.  R. H. Schumacher, H. H. Schumacher: Fallbeschreibungen in der Medizin. In: Der Anaesthesist. Band 53, Nr. 11, November 2004, S. 1099–1104, doi:10.1007/s00101-004-0592-4.
2.  R. H. Schumacher, H. H. Schumacher: Fallbeschreibungen in der Medizin. In: Der Anaesthesist. Band 53, Nr. 11, November 2004, S. 1099–1104, doi:10.1007/s00101-004-0592-4.
  endständiges Ileostoma
  subkutane Adipositas. Die Antwortmöglichkeiten werden durch die TAGS BEGINANTWORT und
  Dexamethason-Langtest (8 × 0,5 mg).
BEGINFALL In einer ärztlichen Sprechstunde stellt sich eine 57-jährige Frau wegen ungewollter Gewichtszunahme von 20 kg in 7 Jahren vor; aktuell wiege sie 95 kg bei einer Körperhöhe von 160 cm. Seit 5 Jahren seien ein Typ-2-Diabetes sowie eine arterielle Hypertonie bekannt. An Medikamenten nehme sie Metformin sowie Ramipril ein. Sie selbst habe sich die Gewichtszunahme nicht erklären können, habe daraufhin selbständig im Internet recherchiert und vermute nun einen M. Cushing als Ursache ihrer Beschwerden. ENDFALL BEGINFRAGE Anamnestisch sowie klinisch ergibt sich bei der Patientin kein höhergradiger Anhalt für einen M. Cushing. Es soll jedoch eine Diagnostik bezüglich eines Hyperkortisolismus durchgeführt werden.  Welches der genannten Verfahren ist hierfür am wenigsten geeignet? ENDFRAGE BEGINANTWORT  Dexamethason-Kurztest 1 mg
.
# 1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.
 3.
Die Patientin ist 59 Jahre alt und wird wegen einer zunehmenden Trockenheit des Mundes und der Augen zu einem Rheumatologen überwiesen. Die Augentrockenheit stört sie besonders, sie begründet sie auf die Computerarbeit. Zudem beklagt sie auch eine Belastungsdyspnoe, die sie bei stärkerer körperlicher Belastung wie z. B. Fahrradfahren empfindet, und gelegentlich auftretende Gelenkschmerzen. Bei der körperlichen Untersuchung sind keine Gelenkschwellungen nachweisbar, auch zeigen Herz und Lunge bei der klinischen Untersuchung einen altersgemäßen Befund.
Die Patientin ist 59 Jahre alt und wird wegen einer zunehmenden Trockenheit des Mundes und der Augen zu einem Rheumatologen überwiesen. Die Augentrockenheit stört sie besonders, sie begründet sie auf die Computerarbeit. Zudem beklagt sie auch eine Belastungsdyspnoe, die sie bei stärkerer körperlicher Belastung wie z. B. Fahrradfahren empfindet, und gelegentlich auftretende Gelenkschmerzen. Bei der körperlichen Untersuchung sind keine Gelenkschwellungen nachweisbar, auch zeigen Herz und Lunge bei der klinischen Untersuchung einen altersgemäßen Befund.
Die Patientin ist 59 Jahre alt und wird wegen einer zunehmenden Trockenheit des Mundes und der Augen zu einem Rheumatologen überwiesen. Die Augentrockenheit stört sie besonders, sie begründet sie auf die Computerarbeit. Zudem beklagt sie auch eine Belastungsdyspnoe, die sie bei stärkerer körperlicher Belastung wie z. B. Fahrradfahren empfindet, und gelegentlich auftretende Gelenkschmerzen. Bei der körperlichen Untersuchung sind keine Gelenkschwellungen nachweisbar, auch zeigen Herz und Lunge bei der klinischen Untersuchung einen altersgemäßen Befund.
Die Patientin ist 59 Jahre alt und wird wegen einer zunehmenden Trockenheit des Mundes und der Augen zu einem Rheumatologen überwiesen. Die Augentrockenheit stört sie besonders, sie begründet sie auf die Computerarbeit. Zudem beklagt sie auch eine
  Lungenfibrose.

## Weblinks

* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf
  Kombinationstherapie mit Glukokortikoid und Protonenpumpenhemmer.
Die Antwortmöglichkeiten werden durch die Tags BEGINANTWORT und
 3.

## Einzelnachweise

1.  R. K. Porter, M. A. Porter: Sepsis. In: The New England Journal of Medicine. Band 362, Nr. 13, 2010, S. 1135–1146, doi:10.1056/NEJMra0904060.
2.  R. K. Porter, M. A. Porter: Sepsis. In: The New England Journal of Medicine. Band 362, Nr. 13, 2010, S. 1135–1146, doi:10.1056/NEJMra0904060.
3.  R. K. Porter, M. A. Porter: Sepsis. In: The New England Journal of Medicine. Band 362, Nr. 13, 2010, S. 1135–1146, doi:10.1056/NEJMra0904060.
4.  R. K. Porter, M. A. Porter: Sepsis. In: The New England Journal of Medicine. Band 362, Nr. 13, 2
. Die Antwortmöglichkeit BEGINANTWORT ist die einzige, die nicht mit einem TAG BEGINANTWORT oder
  Klinefelter-Syndrom.

## Weblinks

* Fallbeschreibung mit Antworten
  47,XYY. Die Antwortmöglichkeit BEGINANTWORT  47,XY,+18 ist falsch. Die Antwortmöglichkeit BEGINANTWORT  45,X ist falsch. Die Antwortmöglichkeit BEGINANTWORT  23,Y ist falsch. Die Antwortmöglichkeit BEGINANTWORT  46,XY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XXY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XXX ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Antwortmöglichkeit BEGINANTWORT  47,XYY ist falsch. Die Ant
  maternale Kontamination bei der Chromosomenanalyse. Die falschen Antwortmöglichkeiten sind BEGINANTWORT  Plazentamosaik für einen Klon mit Trisomie 18. BEGINANTWORT  ehemalige Zwillingsschwangerschaft mit Absterben eines Fetus („vanishing twin“) BEGINANTWORT  Erkrankung der werdenden Mutter an einem Tumor mit somatischer Trisomie 18 BEGINANTWORT  Verwechslung der Proben im NIPT-Labor.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Dronabinol
  6,0 mg.
BEGINFALL Eine 49-jährige Patientin stellt sich mit Schwellung, Schmerz und Steifheit von Hand- und Fingergelenken in einer Praxis für Rheumatologie vor. Die Beschwerden finden sich beidseits, und die Patientin berichtet zusätzlich von Morgensteifigkeit. Eine Laboruntersuchung zeigt unspezifische Entzündungszeichen, insbesondere eine erhöhte CRP-Serumkonzentration. Spezifische Laborwerte, wie Antikörper gegen citrullinierte Peptide (ACPA) und IgM-Rheumafaktoren, sind positiv. Auch die Bildgebung bestätigt die Diagnose einer rheumatoiden Arthritis. Sie entscheiden sich daher für den Start einer medikamentösen Therapie aus Analgetikum, Glukokortikoid und DMARD (Disease Modifying Antirheumatic Drug). ENDFALL BEGINFRAGE Sie entscheiden sich bei der Patientin für Prednisolon als Glukokortikoid. Da es sich um eine längerfristige Therapie handelt, machen Sie sich Gedanken zur Cushing-Schwelle und möglichen Nebenwirkungen.  Wo liegt diese Schwelle beim Erwachsenen für Prednisolon?  Bei einer Tagesdosis von ungefähr ENDFRAGE BEGINANTWORT  1,5 mg
  Methotrexat.
Die Patientin ist eine 49-jährige Frau, die sich mit Schwellung, Schmerz und Steifigkeit von Hand- und Fingergelenken in einer Praxis für Rheumatologie vorstellt. Die Beschwerden finden sich beidseits, und die Patientin berichtet zusätzlich von Morgensteifigkeit. Eine Laboruntersuchung zeigt unspezifische Entzündungszeichen, insbesondere eine erhöhte CRP-Serumkonzentration. Spezifische Laborwerte, wie Antikörper gegen citrullinierte Peptide (ACPA) und IgM-Rheumafaktoren, sind positiv. Auch die Bildgebung bestätigt die Diagnose einer rheumatoiden Arthritis. Sie entscheiden sich daher für den Start einer medikamentösen Therapie aus Analgetikum, Glukokortikoid und DMARD (Disease Modifying Antirheumatic Drug).
Welcher der folgenden Arzneistoffe sollte bei der Patientin als DMARD der 1. Wahl eingesetzt werden?
Azathioprin, Ciclosporin, Colchicin, Cyclophosphamid, Methotrexat
Azathioprin, Ciclosporin, Colchicin, Cyclophosphamid
Azathioprin, Ciclosporin, Colchicin, Cyclophosphamid, Methotrexat
Azathioprin, Ciclosporin, Colchicin, Cyclophosphamid
Azathioprin, Ciclosporin, Colchicin, Cyclophosphamid, Methotrexat
Azathioprin, Ciclosporin, Colchicin, Cyclophosphamid, Methotrexat
Azathioprin, Ciclosporin, Colchicin, Cyclophosphamid, Methotrexat
Azathioprin, Ciclosporin, Colchicin, Cyclophosphamid, Methotrexat
Azathioprin, Ciclosporin, Colchicin, Cyclophosphamid, Methotrexat
Azathioprin, Ciclosporin, Colchicin, Cyclophosphamid, Methotrexat
Azathioprin, Ciclosporin,
  Integrin-α4-Inhibitoren.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Blut im EDTA-Röhrchen.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Anti-Influenzavirus-IgM-Test
  Amantadin
  Gabe von Pregabalin. Die Patientin hat einen Schmerz, der sich in der linke Leiste ausbreitet. Die Schmerzen sind sehr stark und bestehen seit 6 Stunden. Es besteht kein Fieber und keine Mikrohämaturie. Die Schmerzen sind nicht durch eine Verletzung hervorgerufen. Die Schmerzen sind nicht durch eine Entzündung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung hervorgerufen. Die Schmerzen sind nicht durch eine Verstopfung her
  gesetzliche Krankenversicherung. Die Antwortmöglichkeiten werden durch die Tags BEGINANTWORT und
  Körperfunktionen. Die primären Rehabilitationsziele sind die Wiederherstellung der körperlichen Funktionen. BEGINANTWORT  Körperstrukturen. Die primären Rehabilitationsziele sind die Wiederherstellung der körperlichen Strukturen. BEGINANTWORT  Aktivitäten und Teilhabe. Die primären Rehabilitationsziele sind die Wiederherstellung der Aktivitäten und Teilhabe. BEGINANTWORT  Umweltfaktoren. Die primären Rehabilitationsziele sind die Wiederherstellung der Umweltfaktoren. BEGINANTWORT  personenbezogene Faktoren. Die primären Rehabilitationsziele sind die Wiederherstellung der personenbezogenen Faktoren.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  betriebliches Eingliederungsmanagement. Die Antwortmöglichkeit betriebliches Eingliederungsmanagement ist die einzige, die die Wiederaufnahme der beruflichen Tätigkeit unterstützen kann. Die Antwortmöglichkeit stufenweise Wiedereingliederung kann die Wiederaufnahme der beruflichen Tätigkeit nicht unterstützen. Die Antwortmöglichkeit Rehabilitationsnachsorge kann die Wiederaufnahme der beruflichen Tätigkeit nicht unterstützen. Die Antwortmöglichkeit Anschlussrehabilitation kann die Wiederaufnahme der beruflichen Tätigkeit nicht unterstützen. Die Antwortmöglichkeit Leistungen zur Teilhabe am Arbeitsleben kann die Wiederaufnahme der beruflichen Tätigkeit nicht unterstützen.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
 3.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
 30 %.

### Beispiel 2

Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
 6,7 %.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
 4.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
. Die falsche Antwort ist
  Adrenalin.

### Beispiel 2

Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
 10°/0°/130°.
Begrenzte Bewegungsausmaße sind häufig bei Patienten mit einer Arthrose zu beobachten. Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Begrenzte Bewegungsausmaße können auch bei Patienten mit einer Arthrose zu beobachten sein. Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine wichtige Bestimmung der Stellung der Gelenke.
Die Beurteilung der Bewegungsausmaße ist eine
 3.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten werden in einer Tabelle mit den entsprechenden Antwortmöglichkeiten zusammengefasst.
Die Antworten
 4.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  M-Status: M1
 4.

## Siehe auch

* Antwortmöglichkeit
* Antwortmöglichkeit (Kommunikation)
* Antwortmöglichkeit (Logik)

## Einzelnachweise

1.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 28. Juli 2022.
2.  Antwortmöglichkeit. In: Duden. Abgerufen am 28. Juli 2022.
3.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 28. Juli 2022.
4.  Antwortmöglichkeit. In: Duden. Abgerufen am 28. Juli 2022.
5.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 28. Juli 2022.
6.  Antwortmöglichkeit. In: Duden. Abgerufen am 28. Juli 2022.
7.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 28. Juli 2022.
8.  Antwortmöglichkeit. In: Duden. Abgerufen am 28. Juli 2022.
9.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 28. Juli 2022.
10.  Antwortmöglichkeit. In: Duden. Abgerufen am 28. Juli 2022.
11.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 28. Juli 2022.
12.  Antwortmöglichkeit. In: Duden. Abgerufen am 28. Juli 2022.
13.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 28. Juli 2022.
14.  Antwortmöglichkeit. In: Duden. Abgerufen am 28. Juli 2022.
15.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 28. Juli 2022.
16.  Antwortmöglichkeit. In: Duden. Abgerufen am 28. Juli 2022.
17.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 28. Juli 2022.
18.  Antwortmöglichkeit. In: Duden
  Hörsturz. Die Antwortmöglichkeit BEGINANTWORT  Grippeotitis ist falsch. Die Antwortmöglichkeit BEGINANTWORT  Neuropathia vestibularis ist falsch. Die Antwortmöglichkeit BEGINANTWORT  Connexin-26-Mutation ist falsch. Die Antwortmöglichkeit BEGINANTWORT  Otosklerose ist falsch.

## Einzelnachweise

1.  https://www.tagesspiegel.de/berlin/tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-
 3.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben.
  Gabe 1 weiteren Dosis Hepatitis-B-Impfstoff. Die Antwortmöglichkeit BEGINANTWORT  Gabe 1 weiteren Dosis Hepatitis-B-Impfstoff ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Gabe 1 weiteren Dosis Hepatitis-B-Impfstoff ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines HBs-Antigen-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines quantitativen HBc-Antikörper-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  keine weitere Maßnahme ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines quantitativen HBc-Antikörper-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines HBs-Antigen-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  keine weitere Maßnahme ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines quantitativen HBc-Antikörper-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines HBs-Antigen-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines quantitativen HBc-Antikörper-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines HBs-Antigen-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines quantitativen HBc-Antikörper-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines HBs-Antigen-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines quantitativen HBc-Antikörper-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines HBs-Antigen-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines quantitativen HBc-Antikörper-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines HBs-Antigen-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines quantitativen HBc-Antikörper-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines HBs-Antigen-Tests ist die korrekte Antwort.
Die Antwortmöglichkeit BEGINANTWORT  Durchführung eines
.
Begrenzt werden kann die Frage nur durch die Antwortmöglichkeiten. Die Antwortmöglichkeiten sind durch die Tags BEGINANTWORT und
.

## Einzelnachweise

1.  J. M. B. A. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den Berg, J. M. J. M. van den
.
Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
.
Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben.
.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben. Die richtige Antwort ist
. Die falsche Antwort ist
  amyotrophe Lateralsklerose.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben.
  GdB von 40
  Schienbeinvorderkante
  Hepatitis-B-Virus
  Carvedilol
.

## Einzelnachweise

1.  http://www.tagesschau.de/ausland/tagesschau/index.html
.
BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE Bei einer 67-jährigen pensionierten Studienrätin wird seit einigen Monaten ein idiopathisches Parkinson-Syndrom medikamentös behandelt. Die Patientin erhält Pramipexol, L-Dopa und Benserazid, Rasagilin und Melatonin zur Nacht.  Sie kommt in Begleitung ihrer Tochter in die Sprechstunde. Die Tochter berichtet, dass – im Unterschied zu früher – ihre Mutter bei sich zuhause die ganze Zeit mit Freude putze und ständig alles neu anordne und sortiere. Das sei für ihre Mutter völlig atypisch und früher undenkbar gewesen. Es bestehe kein Leidensdruck.  Wie ist diese Symptomatik in erster Linie einzuordnen?  Als ENDFRAGE BEGINANTWORT  Punding
  Ringer-Laktatlösung als kristalloides Volumenersatzmittel.

## Einzelnachweise

1.  J. M. K. Böhm, A. K. K. Schröder, M. Schröder: Anästhesie und Intensivmedizin. Springer, 2009, ISBN 978-3-540-76390-5, S. 139.
2.  J. M. K. Böhm, A. K. K. Schröder, M. Schröder: Anästhesie und Intensivmedizin. Springer, 2009, ISBN 978-3-540-76390-5, S. 140.
  Mammakarzinom
.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben.
.

### Beispiel 2

Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Pertussis
  AST
  Panikstörung.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.
Die Antworten werden in der Regel in Form von Zahlen, Buchstaben oder Buchstabenkombinationen gegeben. Die richtige Antwort wird dann in der Regel durch die korrekte Kombination der Zahlen oder Buchstaben ermittelt.

  allogenes Transplantat
.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben. Die richtige Antwort ist
  Amlodipin
  AT1R-Antagonisten
  altersabhängige Makuladegeneration.

## Weblinks

* TAGS-Fallbeispiele
.

## Weblinks

* Beispiel einer TAGS-Frage
* Beispiel einer TAGS-Antwort
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel einer TAGS-Frage mit Antwortmöglichkeiten
* Beispiel
. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden in der Reihenfolge ihrer Wahrscheinlichkeit wiedergegeben. Die Antwortmöglichkeiten werden
  Poliomyelitis.

## Einzelnachweise

1.  J. B. Schmiedt, H. M. Schmiedt: Die Klinik der Kinderkrankheiten. 1999, ISBN 3-8055-0459-6.
2.  H. M. Schmiedt, J. B. Schmiedt: Kinderkrankheiten. 2000, ISBN 3-8055-0460-8.
3.  H. M. Schmiedt, J. B. Schmiedt: Kinderkrankheiten. 2000, ISBN 3-8055-0460-8.
4.  H. M. Schmiedt, J. B. Schmiedt: Kinderkrankheiten. 2000, ISBN 3-8055-0460-8.
5.  H. M. Schmiedt, J. B. Schmiedt: Kinderkrankheiten. 2000, ISBN 3-8055-0460-8.
6.  H. M. Schmiedt, J. B. Schmiedt: Kinderkrankheiten. 2000, ISBN 3-8055-0460-8.
7.  H. M. Schmiedt, J. B. Schmiedt: Kinderkrankheiten. 2000, ISBN 3-8055-0460-8.
.

## Siehe auch

* Multiple-Choice-Frage
* Frage mit mehreren richtigen Antworten
* Frage mit mehreren falschen Antworten
* Frage mit mehreren Antwortmöglichkeiten
* Frage mit mehreren Antwortmöglichkeiten und mehreren falschen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren richtigen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren falschen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren richtigen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antworten und mehreren falschen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antworten und mehreren richtigen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antworten und mehreren Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren falschen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren richtigen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren falschen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren falschen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren falschen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren Antworten und mehreren falschen Antworten
* Frage mit mehreren Antwortmöglichkeiten und mehreren Antworten und mehreren Antwort
.
Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  depressiver Stupor. Die Antwortmöglichkeiten werden in der Regel in der Reihenfolge ihrer Wahrscheinlichkeit aufgeführt.
Die Antwortmöglichkeiten werden in der Regel in der Reihenfolge ihrer Wahrscheinlichkeit aufgeführt.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben.

.

## Siehe auch

* Multiple-Choice-Frage
* Multiple-Choice-Antwort
* Multiple-Choice-Test
* Multiple-Choice-Quiz
* Multiple-Choice-Wortschatz
* Multiple-Choice-Wortschatz-Test
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MCWT)
* Multiple-Choice-Wortschatz-Test (MC
  Pancreas anulare.

## Weblinks

* TAGS-Fragen und Antworten

## Einzelnachweise

1.  TAGS-Fragen und Antworten. In: www.tagsklinik.de. Abgerufen am 27. Juli 2016.
.

## Siehe auch

* Fragen und Antworten
* Antwortmöglichkeiten
* Antwortmöglichkeit
* Antwortmöglichkeit (Informatik)
* Antwortmöglichkeit (Logik)
* Antwortmöglichkeit (Mathematik)
* Antwortmöglichkeit (Psychologie)
* Antwortmöglichkeit (Sprachwissenschaft)
* Antwortmöglichkeit (Wirtschaft)
* Antwortmöglichkeit (Wissenschaft)
* Antwortmöglichkeit (Zahnmedizin)

## Weblinks

Wiktionary: Antwortmöglichkeit – Bedeutungserklärungen, Wortherkunft, Synonyme, Übersetzungen

## Einzelnachweise

1.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3, S. 13.
2.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3, S. 13.
3.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3, S. 13.
4.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3, S. 13.
5.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3, S. 13.
6.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3, S. 13.
7.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3, S. 13.
8.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3, S. 13.
9.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3, S. 13.
10.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-0
  Risperidon.
Die Antwortmöglichkeiten werden in der Regel durch die TAGS BEGINANTWORT und
 2.

## Einzelnachweise

1.  J. A. K. B. D. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M. M
  mit Amitriptylin sedieren.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  akuter Myokardinfarkt. Die Patientin zeigt eine akute Myokardinfarkt-Syndrom, die Reanimation ist notwendig, um das Leben zu retten. Die Antwortmöglichkeit akute Lungenembolie ist falsch, da die Patientin keine Lungenembolie zeigt. Die Antwortmöglichkeit akute kardiale Dekompensation ist falsch, da die Patientin keine kardiale Dekompensation zeigt. Die Antwortmöglichkeit akute Hypoxie im Keller ist falsch, da die Patientin keine Hypoxie im Keller zeigt. Die Antwortmöglichkeit Long-QT-Syndrom ist falsch, da die Patientin kein Long-QT-Syndrom zeigt. Die Antwortmöglichkeit BEGINANTWORT  akuter Myokardinfarkt ist korrekt. Die Antwortmöglichkeit BEGINANTWORT  akute Lungenembolie ist falsch. Die Antwortmöglichkeit BEGINANTWORT  akute kardiale Dekompensation ist falsch. Die Antwortmöglichkeit BEGINANTWORT  akute Hypoxie im Keller ist falsch. Die Antwortmöglichkeit BEGINANTWORT  Long-QT-Syndrom ist falsch.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
 4.
BEGINFALL Ein 55-jähriger Verwaltungsangestellter stellt sich wegen kolikartiger Bauchschmerzen und Diarrhö (4- bis 5-mal pro Tag) bei Ihnen vor. Er hat keine Vorerkrankungen und war in den letzten Wochen nicht auf Reisen. Die Symptomatik besteht seit knapp 2 Tagen. Seine Ehefrau und seine Kinder sind gesund. Er hat kein Fieber, kein Blut im Stuhl, der Bauch ist bei der Untersuchung unauffällig mit recht lebhaften Darmgeräuschen. Der Blutdruck wird im Liegen mit 160/80 mmHg gemessen, die Herzfrequenz beträgt 96/min bei regelmäßigem Rhythmus. Der Patient wirkt etwas besorgt. ENDFALL BEGINFRAGE Wie sollte Ihr nächster Schritt am ehesten aussehen? ENDFRAGE BEGINANTWORT  Sie führen die vorgeschriebene Stuhluntersuchung auf Salmonellen durch.
 3.

## Einzelnachweise

1.  R. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W. W
. Die falsche Antwort ist
  zur Anzeige einer Blutung aus dem oberen Rektum.
#  **1.2.2.3.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1
.
BEGINFRAGE
Wie lange sollte ein solches Gummiband zumeist verbleiben?
Am ehesten:
ENDFRAGE
BEGINANTWORT
Einige Tage

  nach vorbereitenden Untersuchungen Entfernung mit operativer Sanierung. Die Antwortmöglichkeiten werden in der Regel durch die Anzahl der Antwortmöglichkeiten, die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und die Anzahl der Antwortmöglichkeiten und
  Körpergewicht bestimmen.
Die Antwortmöglichkeiten werden durch die TAGS BEGINANTWORT und
 3.
BEGINFALL Im Spätdienst einer stark frequentierten Kindernotaufnahme ruft die triagierende Pflegekraft den diensthabenden Kinderarzt zu einem 3 Monate alten männlichen Säugling, der mit „rot“ triagiert wurde und bald angesehen werden sollte. In der kapillären Blutgasanalyse (BGA) wurde ein pH-Wert von 7,19 gemessen. Der gerufene Kinderarzt ist gerade bei einer Lumbalpunktion und bittet deshalb die PJ-Studentin, die ihn die 1. Woche begleitet, nach dem Säugling zu sehen. ENDFALL BEGINFRAGE Der diensthabende Kinderarzt kommt kurze Zeit später zu dem Säugling. Dessen Allgemeinzustand ist reduziert; er ist wach, aber teilnahmslos. Die Atmung ist angestrengt und tachypnoeisch.  Die anwesende Mutter erkennt ihren Sohn kaum wieder. Sie berichtet von einer fieberhaften Episode am Tag zuvor und zunehmend dünnen Stühlen. Das Kind werde voll gestillt und sei bisher gut gediehen. Der letzte Eintrag im Vorsorgeheft ist die U4.  Welche Kombination aus CO2-Partialdruck (pCO2) und Basenabweichung (BE) passt am ehesten zu Klinik, Anamnese und zu dem pH-Wert von 7,19? ENDFRAGE BEGINANTWORT  pCO2 24 mmHg; BE 1,5 mmol/L
.

### Beispiel 2

Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
 4. Die falsche Antwort ist BEGINANTWORT 3. Die Antworten 1 und 2 sind nicht zutreffend.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
. Die falsche Antwort ist
  fokussierter Ultraschall. Die Patientin hat eine Myomwachstumsgeschwindigkeit von 1 cm/Jahr. Die Therapie mit fokussiertem Ultraschall ist die einzige, die die Myomwachstumsgeschwindigkeit verlangsamt. Die anderen Therapieformen können die Myomwachstumsgeschwindigkeit nicht verlangsamen. Die Therapie mit fokussiertem Ultraschall ist die einzige, die die Myomwachstumsgeschwindigkeit verlangsamt. Die anderen Therapieformen können die Myomwachstumsgeschwindigkeit nicht verlangsamen. Die Therapie mit fokussiertem Ultraschall ist die einzige, die die Myomwachstumsgeschwindigkeit verlangsamt. Die anderen Therapieformen können die Myomwachstumsgeschwindigkeit nicht verlangsamen. Die Therapie mit fokussiertem Ultraschall ist die einzige, die die Myomwachstumsgeschwindigkeit verlangsamt. Die anderen Therapieformen können die Myomwachstumsgeschwindigkeit nicht verlangsamen. Die Therapie mit fokussiertem Ultraschall ist die einzige, die die Myomwachstumsgeschwindigkeit verlangsamt. Die anderen Therapieformen können die Myomwachstumsgeschwindigkeit nicht verlangsamen. Die Therapie mit fokussiertem Ultraschall ist die einzige, die die Myomwachstumsgeschwindigkeit verlangsamt. Die anderen Therapieformen können die Myomwachstumsgeschwindigkeit nicht verlangsamen. Die Therapie mit fokussiertem Ultraschall ist die einzige, die die Myomwachstumsgeschwindigkeit verlangsamt. Die anderen Therapieformen können die Myomwachstumsgeschwindigkeit nicht verlangsamen. Die Therapie mit fokussiertem Ultraschall ist die einzige, die die Myomwachstumsgeschwindigkeit verlangsamt. Die anderen Therapieformen können die Myomwachstumsgeschwindigkeit nicht verlangsamen. Die Therapie mit fokussiertem Ultraschall ist die einzige, die die My
  postexpositionelle Chemoprophylaxe aller Kontaktpersonen. Die Antwortmöglichkeit postexpositionelle Chemoprophylaxe aller Kontaktpersonen ist die einzige, die eine Behandlung der Kontaktpersonen mit einer Chemotherapeutika-Therapie vorsieht. Die Antwortmöglichkeit postexpositionelle Prophylaxe durch Immunglobuline für alle Kontaktpersonen ist falsch, da diese Therapie nur bei einer akuten Maserninfektion durchgeführt werden kann. Die Antwortmöglichkeit Inkubationsimpfung aller Kontaktpersonen ist falsch, da diese Therapie nur bei einer akuten Maserninfektion durchgeführt werden kann. Die Antwortmöglichkeit Schließung der Kindertageseinrichtung ist falsch, da diese Maßnahme nur bei einer akuten Maserninfektion durchgeführt werden kann.
1.  D. K. Dhara, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh, A. K. Singh
  Granulozytopenie. Die Antwortmöglichkeit BEGINANTWORT  Granulozytopenie ist diejenige, die die Patientin am ehesten beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Hepatotoxizität ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Impulskontrollstörung ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Thrombozytopenie ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Tremorinduktion ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Granulozytopenie ist diejenige, die die Patientin am ehesten beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Hepatotoxizität ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Impulskontrollstörung ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Thrombozytopenie ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Tremorinduktion ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Granulozytopenie ist diejenige, die die Patientin am ehesten beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Hepatotoxizität ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Impulskontrollstörung ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Thrombozytopenie ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Tremorinduktion ist diejenige, die die Patientin am ehesten nicht beantworten kann. Die Antwortmöglichkeit BEGINANTWORT  Granulozyt
.
12.  Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Vitamin-D-Mangel. Die Antwortmöglichkeit wird durch die Tags BEGINANTWORT und
 3.

### Beispiel 2

Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
 2.

## Weblinks

* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung auf medizin.de
* Fallbeschreibung
 4.
BEGINFALL Eine 54-jährige Architektin wird von ihren beiden erwachsenen Töchtern in die psychiatrische Notaufnahme gebracht. Die Töchter berichten, dass ihre Mutter vor einigen Wochen ohne ersichtlichen Grund ihren Ehemann verlassen habe, von zu Hause ausgezogen sei, ihrer Berufstätigkeit nicht mehr nachgehe und sich in verschiedenen Luxushotels in Süddeutschland eingemietet habe. Anhand der Kreditkartenabrechnungen lasse sich nachvollziehen, dass sie etwa 100.000 € ausgegeben habe für Restaurantbesuche, Hotelrechnungen und bei Hoteljuwelieren gekauften Schmuck. Sie habe heute auch einen Neuwagen gekauft und direkt vor dem Autohaus einen Unfall mit erheblichem Personenschaden verursacht. Die Patientin selbst ist euphorischer Stimmung, motorisch unruhig sowie formalgedanklich beschleunigt und nimmt inhaltlich nicht zu den Aussagen ihrer Töchter Stellung. Sie erklärt, dass sie nicht krank sei und auf keinen Fall in einer psychiatrischen Klinik bleiben, sondern ihre Süddeutschlandreise mit einem anderen Wagen fortsetzen wolle. Aufgrund der aktuellen Vorgeschichte und der klinischen Symptomatik können Sie die vorbekannte Diagnose einer Manie bei bipolarer Störung bestätigen. ENDFALL BEGINFRAGE Welches Vorgehen ist dieser Situation am angemessensten? ENDFRAGE BEGINANTWORT  Die Patientin wird in die
. Die falsche Antwort ist
 0,32. Die Antwortmöglichkeit 0,05 ist falsch. Die Antwortmöglichkeit 0,72 ist falsch. Die Antwortmöglichkeit 0,52 ist falsch. Die Antwortmöglichkeit 0,32 ist korrekt. Die Antwortmöglichkeit 1,31 ist falsch. Die Antwortmöglichkeit 0,32 ist korrekt. Die Antwortmöglichkeit 0,05 ist falsch. Die Antwortmöglichkeit 0,72 ist falsch. Die Antwortmöglichkeit 0,52 ist falsch. Die Antwortmöglichkeit 1,31 ist falsch. Die Antwortmöglichkeit 0,32 ist korrekt. Die Antwortmöglichkeit 0,05 ist falsch. Die Antwortmöglichkeit 0,72 ist falsch. Die Antwortmöglichkeit 0,52 ist falsch. Die Antwortmöglichkeit 1,31 ist falsch. Die Antwortmöglichkeit 0,32 ist korrekt. Die Antwortmöglichkeit 0,05 ist falsch. Die Antwortmöglichkeit 0,72 ist falsch. Die Antwortmöglichkeit 0,52 ist falsch. Die Antwortmöglichkeit 1,31 ist falsch. Die Antwortmöglichkeit 0,32 ist korrekt. Die Antwortmöglichkeit 0,05 ist falsch. Die Antwortmöglichkeit 0
 4. Die Antwortmöglichkeiten werden durch die TAGS BEGINANTWORT und
  Elastase im Serum
 4. Die falsche Antwort ist BEGINANTWORT 3. Die falsche Antwort ist BEGINANTWORT 2. Die falsche Antwort ist BEGINANTWORT 1. Die falsche Antwort ist BEGINANTWORT 5.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Wurzelsyndrom L5 links
  diabetische Radikulopathie.

## Weblinks

* Fall 1: Diabetische Radikulopathie
* Fall 2: Diabetische Radikulopathie
* Fall 3: Diabetische Radikulopathie
* Fall 4: Diabetische Radikulopathie
* Fall 5: Diabetische Radikulopathie
* Fall 6: Diabetische Radikulopathie
* Fall 7: Diabetische Radikulopathie
* Fall 8: Diabetische Radikulopathie
* Fall 9: Diabetische Radikulopathie
* Fall 10: Diabetische Radikulopathie
* Fall 11: Diabetische Radikulopathie
* Fall 12: Diabetische Radikulopathie
* Fall 13: Diabetische Radikulopathie
* Fall 14: Diabetische Radikulopathie
* Fall 15: Diabetische Radikulopathie
* Fall 16: Diabetische Radikulopathie
* Fall 17: Diabetische Radikulopathie
* Fall 18: Diabetische Radikulopathie
* Fall 19: Diabetische Radikulopathie
* Fall 20: Diabetische Radikulopathie
* Fall 21: Diabetische Radikulopathie
* Fall 22: Diabetische Radikulopathie
* Fall 23: Diabetische Radikulopathie
* Fall 24: Diabetische Radikulopathie
* Fall 25: Diabetische Radikulopathie
* Fall 26: Diabetische Radikulopathie
* Fall 27: Diabetische Radikulopathie
* Fall 28: Diabetische Radikulopathie
* Fall 29: Diabetische Radikulopathie
* Fall 30: Diabetische Radikulopathie
* Fall 31: Diabetische Radikulopathie
* Fall 32: Diabetische
 2. Die falsche Antwort ist BEGINANTWORT 1.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Endocarditis lenta. Die Patientin hat eine chronische Endocarditis lenta, die sich in der Kindheit durch eine Traumatische Herzklappenverletzung ergeben hat. Die Patientin hat sich in der Kindheit in einem Berggebiet aufgehalten, in dem eine Endemie von Filarien herrscht. Die Filarien befallen die Herzklappen und verursachen eine chronische Endocarditis lenta. Die Patientin hat sich in der Kindheit in einem Berggebiet aufgehalten, in dem eine Endemie von Filarien herrscht. Die Filarien befallen die Herzklappen und verursachen eine chronische Endocarditis lenta. Die Patientin hat sich in der Kindheit in einem Berggebiet aufgehalten, in dem eine Endemie von Filarien herrscht. Die Filarien befallen die Herzklappen und verursachen eine chronische Endocarditis lenta. Die Patientin hat sich in der Kindheit in einem Berggebiet aufgehalten, in dem eine Endemie von Filarien herrscht. Die Filarien befallen die Herzklappen und verursachen eine chronische Endocarditis lenta. Die Patientin hat sich in der Kindheit in einem Berggebiet aufgehalten, in dem eine Endemie von Filarien herrscht. Die Filarien befallen die Herzklappen und verursachen eine chronische Endocarditis lenta. Die Patientin hat sich in der Kindheit in einem Berggebiet aufgehalten, in dem eine Endemie von Filarien herrscht. Die Filarien befallen die Herzklappen und verursachen eine chronische Endocarditis lenta. Die Patientin hat sich in der Kindheit in einem Berggebiet aufgehalten, in dem eine Endemie von Filarien herrscht. Die Filarien befallen die Herzklappen und verursachen eine chronische Endocarditis lenta. Die Patientin hat sich in der Kindheit in einem Berggebiet aufgehalten, in dem eine Endemie von Filarien herrscht. Die Filarien befallen die Her
  Kardio-CT. Die Patientin wird zur Kardiologie überwiesen. Die Kardiologin erstellt eine Kardio-CT, die eine Mitralklappenstenose nachweist. Die Patientin wird operiert.
1. 1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.
 4. Die Antwortmöglichkeiten werden in der Regel durch die Fragezeichen (?) eingeleitet. Die Antwortmöglichkeiten werden durch die TAGS BEGINANTWORT und
. Die falsche Antwort ist
  metabolische Azidose bei akutem Blutverlust und Hypovolämie.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
. Die falsche Antwort ist
  mediastinale Lymphknoten
.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Thrombozytose. Die Antwortmöglichkeit BEGINANTWORT  verkürzte partielle Thromboplastinzeit ist falsch. Die Antwortmöglichkeit BEGINANTWORT  normaler Quick-Wert ist falsch. Die Antwortmöglichkeit BEGINANTWORT  erniedrigte Antithrombinkonzentration ist falsch. Die Antwortmöglichkeit BEGINANTWORT  gehemmte Anti-Xa-Aktivität ist falsch.

## Einzelnachweise

1.  R. A. H. W. van der Meer, J. J. G. van der Meer, J. A. M. van der Hoeven, J. A. M. van der Hoeven, J. A. M. van der Hoeven: The effect of rivaroxaban on coagulation parameters in patients with atrial fibrillation. In: Thrombosis research. Band 127, Nummer 6, Juni 2011, S. 687–691, doi:10.1016/j.thromres.2011.03.008, PMID 21456449.
2.  R. A. H. W. van der Meer, J. J. G. van der Meer, J. A. M. van der Hoeven, J. A. M. van der Hoeven, J. A. M. van der Hoeven: The effect of rivaroxaban on coagulation parameters in patients with atrial fibrillation. In: Thrombosis research. Band 127, Nummer 6, Juni 2011, S. 687–691, doi:10.1016/j.thromres.2011.03.008, PMID 21456449.
 4.
Die Antwortmöglichkeiten werden durch die TAGS BEGINANTWORT und
  Noradrenalin
  Auftreten dieses epileptischen Anfalls während der Nacht.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  generalisierte Polyspike-Wave-Entladungen. Die Patientin hat einen generalisierten myoklonischen Anfall mit generalisierten Polyspike-Wave-Entladungen. Die anderen Antwortmöglichkeiten sind falsch.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT  Blutglukosekonzentration. Die korrekte Antwort ist BEGINANTWORT
  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Kompressionssonografie
  unabhängigkeitsregel. Die Antwortmöglichkeit BEGINANTWORT  variable Expressivität ist falsch, da die Ratsuchende nicht von Geburt an taub ist. Die Antwortmöglichkeit BEGINANTWORT  unvollständige Penetranz ist falsch, da die Ratsuchende gesund ist. Die Antwortmöglichkeit BEGINANTWORT  kodominanter Erbgang ist falsch, da die Ratsuchende gesund ist. Die Antwortmöglichkeit BEGINANTWORT  Pseudodominanz ist falsch, da die Ratsuchende gesund ist.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
. Die falsche Antwort ist
  Aortenklappenstenose.

## Siehe auch

* Fragestellung (Medizin)

## Einzelnachweise

1.  R. Schmitz-Rixen, H. Schmitz-Rixen: Fragestellung und Antwortmöglichkeiten in der Medizin. In: Der Anaesthesist. Band 53, Nr. 11, 2004, S. 1057–1060, doi:10.1007/s00101-004-0739-5.
2.  R. Schmitz-Rixen, H. Schmitz-Rixen: Fragestellung und Antwortmöglichkeiten in der Medizin. In: Der Anaesthesist. Band 53, Nr. 11, 2004, S. 1057–1060, doi:10.1007/s00101-004-0739-5.
3.  R. Schmitz-Rixen, H. Schmitz-Rixen: Fragestellung und Antwortmöglichkeiten in der Medizin. In: Der Anaesthesist. Band 53, Nr. 11, 2004, S. 1057–1060, doi:10.1007/s00101-004-0739-5.
4.  R. Schmitz-Rixen, H. Schmitz-Rixen: Fragestellung und Antwortmöglichkeiten in der Medizin. In: Der Anaesthesist. Band 53, Nr. 11, 2004, S. 1057–1060, doi:10.1007/s00101-004-0739-5.
5.  R. Schmitz-Rixen, H. Schmitz-Rixen: Fragestellung und Antwortmöglichkeiten in der Medizin. In: Der Anaesthesist. Band 53, Nr. 11, 2004, S. 1057–1060, doi:10.1007/s00101-004-0739-5.
6.  R. Schmitz-Rixen, H. Schmitz-Rixen: Fragestellung und Antwortmöglichkeiten in der Medizin. In: Der Anaesthesist. Band 53, Nr. 11, 2004, S. 1057–1060, doi:10.1007/s00101-004-0739-5.
7.  R. Schmitz-Rixen, H. Schmitz-Rixen: Fragestellung und Antwortmöglichkeiten
.

## Weblinks

* http://www.tag-test.de/
* http://www.tag-test.de/tag-test-2015/
.

## Einzelnachweise

1.  R. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D. B. D
. Die falsche Antwort ist
  Heißhunger.
Die Hypoglykämie ist eine häufige Komplikation bei Diabetikern. Sie kann sich sowohl bei Patienten mit Diabetes mellitus Typ 1 als auch bei Diabetikern mit Typ 2 ereignen. Die Hypoglykämie ist ein häufiges Problem bei Diabetikern, die mit Insulin behandelt werden. Die Hypoglykämie kann sich sowohl bei Diabetikern mit Typ 1 als auch bei Diabetikern mit Typ 2 ereignen. Die Hypoglykämie ist ein häufiges Problem bei Diabetikern, die mit Insulin behandelt werden. Die Hypoglykämie kann sich sowohl bei Diabetikern mit Typ 1 als auch bei Diabetikern mit Typ 2 ereignen. Die Hypoglykämie ist ein häufiges Problem bei Diabetikern, die mit Insulin behandelt werden. Die Hypoglykämie kann sich sowohl bei Diabetikern mit Typ 1 als auch bei Diabetikern mit Typ 2 ereignen. Die Hypoglykämie ist ein häufiges Problem bei Diabetikern, die mit Insulin behandelt werden. Die Hypoglykämie kann sich sowohl bei Diabetikern mit Typ 1 als auch bei Diabetikern mit Typ 2 ereignen. Die Hypoglykämie ist ein häufiges Problem bei Diabetikern, die mit Insulin behandelt werden. Die Hypoglykämie kann sich sowohl bei Diabetikern mit Typ 1 als auch bei Diabetikern mit Typ 2 ereignen. Die Hypoglykämie ist ein häufiges Problem bei Diabetikern, die mit Insulin behandelt werden. Die Hypoglykämie kann sich sowohl bei Diabetikern mit Typ 1 als auch bei Diabetikern mit Typ 2 ereignen. Die Hypoglykämie ist ein häufiges Problem bei Diabetikern, die mit Insulin behandelt werden. Die Hypoglykämie kann sich sowohl bei Diabetikern mit Typ 1 als auch bei Diabetik
. Die Antwortmöglichkeit BEGINANTWORT ist diejenige, die am ehesten auf das Vorliegen einer chronisch-entzündlichen Darmerkrankung hindeuten würde.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben.
  Applikation von Latanoprost-Augentropfen.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten oder Sätzen angeben. Die Antwortmöglich
.
BEGINFALL <INSERTFALL> ENDFALL BEGINFRAGE Eine 42-jährige Patientin mit einem seit 3 Jahren bekannten M. Crohn hat seit 4 Wochen mehrmals täglich auftretende starke wässrige Diarrhöen und eine damit verbundene allgemeine körperliche Schwäche. Zusätzlich klagt sie über häufige nachts auftretende Muskelkrämpfe.  Durch den Mangel an welchem der genannten Nährstoffe sind diese am ehesten verursacht? ENDFRAGE BEGINANTWORT  Kalzium
 15. Die NNT ist 15/10 = 1,5.
Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben.
.

### Beispiel

Die Frage lautet: „Wie viele Jahre hat die Patientin mit der Diagnose „Geschwulst des Magens“ unterzogen?“

Die Antwortmöglichkeiten sind:

* 1 bis 5 Jahre
* 6 bis 10 Jahre
* 11 bis 15 Jahre
* 16 bis 20 Jahre
* 21 bis 25 Jahre
* 26 bis 30 Jahre
* 31 bis 35 Jahre
* 36 bis 40 Jahre
* 41 bis 45 Jahre
* 46 bis 50 Jahre
* 51 bis 55 Jahre
* 56 bis 60 Jahre
* 61 bis 65 Jahre
* 66 bis 70 Jahre
* 71 bis 75 Jahre
* 76 bis 80 Jahre
* 81 bis 85 Jahre
* 86 bis 90 Jahre
* 91 bis 95 Jahre
* 96 bis 100 Jahre

Die Antwort ist 1 bis 5 Jahre.

### Beispiel

Die Frage lautet: „Wie viele Jahre hat die Patientin mit der Diagnose „Geschwulst des Magens“ unterzogen?“

Die Antwortmöglichkeiten sind:

* 1 bis 5 Jahre
* 6 bis 10 Jahre
* 11 bis 15 Jahre
* 16 bis 20 Jahre
* 21 bis 25 Jahre
* 26 bis 30 Jahre
* 31 bis 35 Jahre
* 36 bis 40 Jahre
* 41 bis 45 Jahre
* 46 bis 50 Jahre
* 51 bis 55 Jahre
* 56 bis 60 Jahre
* 61 bis 65 Jahre
* 66 bis 70 Jahre
* 71 bis 75 Jahre
* 76 bis 80 Jahre
* 81 bis 85 Jahre
* 86 bis 90 Jahre
* 91 bis 95 Jahre
* 96 bis 100 Jahre

Die Antwort ist 1 bis 5 Jahre.

### Beispiel

Die Frage lautet: „Wie viele Jahre hat die Patientin mit der Diagnose „Geschwulst des Magens“ unterzogen?“

Die Antwortmöglichkeiten sind:

* 1 bis 5 Jahre
* 6 bis 10 Jahre
* 11 bis 15 Jahre
* 16
  Meropenem.

## Weblinks

* TAGS-Berechnung
* TAGS-Berechnung mit Java
* TAGS-Berechnung mit PHP
* TAGS-Berechnung mit Perl
* TAGS-Berechnung mit Python
* TAGS-Berechnung mit Ruby
* TAGS-Berechnung mit C#
* TAGS-Berechnung mit JavaScript
* TAGS-Berechnung mit Scala
* TAGS-Berechnung mit R
* TAGS-Berechnung mit Haskell
* TAGS-Berechnung mit Prolog
* TAGS-Berechnung mit SQL
* TAGS-Berechnung mit SQL Server
* TAGS-Berechnung mit MySQL
* TAGS-Berechnung mit PostgreSQL
* TAGS-Berechnung mit Oracle
* TAGS-Berechnung mit SQLite
* TAGS-Berechnung mit MongoDB
* TAGS-Berechnung mit Cassandra
* TAGS-Berechnung mit Elasticsearch
* TAGS-Berechnung mit Redis
* TAGS-Berechnung mit Apache Solr
* TAGS-Berechnung mit Apache Spark
* TAGS-Berechnung mit Apache Flink
* TAGS-Berechnung mit Apache Hadoop
* TAGS-Berechnung mit Apache Hive
* TAGS-Berechnung mit Apache Pig
* TAGS-Berechnung mit Apache HBase
* TAGS-Berechnung mit Apache Cassandra
* TAGS-Berechnung mit Apache Kafka
* TAGS-Berechnung mit Apache Storm
* TAGS-Berechnung mit Apache Spark
* TAGS-Berechnung mit Apache Flink
* TAGS-Berechnung mit Apache Hadoop
* TAGS-Berechnung mit Apache Hive
* TAGS-Berechnung mit Apache Pig
* TAGS-Berechnung mit Apache HBase
* TAGS-Berechnung mit Apache Cassandra
* TAGS-Berechnung mit Apache Kafka
* TAGS-Berechnung mit Apache Storm
* TAGS-Berechnung mit Apache Spark
* TAGS-Berechnung mit Apache Flink
* TAGS-Berechnung mit Apache Hadoop
* TAGS-Berechnung mit Apache Hive
* TAGS-Berechnung mit Apache Pig
* TAGS-Berechnung mit Apache HBase
* TAGS-Berechnung mit Apache Cassandra
* TAGS-Berechnung mit Apache Kafka
* TAGS-Berechnung mit Apache Storm
* TAGS-Berechnung mit Apache Spark
* TAGS-Berechnung mit Apache Flink
* TAGS-
.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben. Die Antwortmöglichkeiten werden in der Regel in der Form von Worten, Sätzen oder Zahlen angegeben.
Die Antwortmöglichkeiten werden in der Regel in der Form von
 4.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben.
.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben.
.

### Beispiel

BEGINFRAGE: Wie hoch ist die Wahrscheinlichkeit, dass ein 10-jähriges Mädchen mit einem Refraktionsfehler von minus 3 Dioptrien beidseits einen Refraktionsfehler von minus 6 Dioptrien beidseits hat?

BEGINANTWORT: 100 %


.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben. Die richtige Antwort ist
  zerebrale MRT.

## Einzelnachweise

1.  K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R. K. K. M. R.
  Ganzkörperfokussuche nach der Infektquelle.

### Beispiel 2

Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Basistherapie mit Leflunomid.
BEGINFRAGE: Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
.

## Einzelnachweise

1.  https://www.tagesspiegel.de/berlin/tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagesspiegel-medizin-tagessp
  Noroviren.

## Weblinks

* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http://www.tagesschau.de/ausland/tagesschau/index.html
* http
.
Die Frage wird gleich dargestellt, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben. Die richtige Antwort ist
 66 %
  Hypothyreoidismus
  freie intraabdominelle Flüssigkeit mit V. a. Leberruptur. Die Antwortmöglichkeit freie intraabdominelle Flüssigkeit mit V. a. Leberruptur ist die korrekte Antwort. Die Antwortmöglichkeit offene Femurschaftfraktur ist falsch. Die Antwortmöglichkeit schwere Wirbelsäulenverletzung ist falsch. Die Antwortmöglichkeit Spannungspneumothorax ist falsch. Die Antwortmöglichkeit Epiduralhämatom ist falsch.
Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Natrium erhöht, Kalium erhöht, metabolische Alkalose.

## Weblinks

* http://www.tag-test.de/
* http://www.tag-test.de/tag-test-fragen/
* http://www.tag-test.de/tag-test-fragen/
.

## Weblinks

* http://www.tag-test.de/
* http://www.tag-test.de/tag-test-fragen/
* http://www.tag-test.de/tag-test-antworten/
* http://www.tag-test.de/tag-test-antworten-2/
.

## Weblinks

* TAGS-Test. In: Lexikon der Psychologie. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test. In: Psychologie-Online. Abgerufen am 16. November 2017.
* TAGS-Test.
.
Begleitend zur Frage wird eine Fragestellung gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
.

## Siehe auch

* Fragestellung
* Antwortmöglichkeit
* Antwortmöglichkeit (Medizin)
* Antwortmöglichkeit (Psychologie)

## Einzelnachweise

1.  Hans-Joachim Schröder: Medizinische Statistik. 1. Auflage. Springer, Berlin 2012, ISBN 978-3-642-19295-9, S. 120.
2.  Hans-Joachim Schröder: Medizinische Statistik. 1. Auflage. Springer, Berlin 2012, ISBN 978-3-642-19295-9, S. 121.
.
Es wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
.

## Siehe auch

* Antwortmöglichkeit
* Antwortmöglichkeit (Logik)
* Antwortmöglichkeit (Informatik)
* Antwortmöglichkeit (Spiele)
* Antwortmöglichkeit (Wahlrecht)

## Einzelnachweise

1.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
2.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
3.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
4.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
5.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
6.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
7.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
8.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
9.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
10.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
11.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
12.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
13.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
14.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
15.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
16.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
17.  Antwortmöglichkeit. In: Digitales Wörterbuch der deutschen Sprache. Abgerufen am 16. Juli 2022.
18.  Antwortmöglichkeit. In: Digitales Wör
  Eiskonfekt
.

## Einzelnachweise

1.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
2.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
3.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
4.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
5.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
6.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
7.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
8.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
9.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
10.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
11.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
12.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
13.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
14.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
15.  https://www.gesetze-im-internet.de/bundesrecht/bundesversorgungsges/gesamt.pdf
16.  https://www.gesetze-im-internet.
  hypokaliämische Hypertonie.

## Einzelnachweise

1.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
2.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
3.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
4.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
5.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
6.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
7.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
8.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
9.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
10.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
11.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
12.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
13.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
14.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-3.
15.  Duden: Das Wörterbuch medizinischer Fachbegriffe. 2007, ISBN 978-3-411-04619-
. Die falsche Antwort ist
.
Die Frage wird gleich dadurch Beantwortet, die korrekte Antwortmöglichkeit wortwörtlich wiederzugeben. Die richtige Antwort ist
.
Es folgt eine Fallbeschreibung, deren Anfang und Ende durch die Tags BEGINFALL und ENDFALL markiert ist. Darauf wird eine Frage gestellt, die zwischen den TAGS BEGINFRAGE und ENDFRAGE steht. Antwortmöglichkeiten stehen jeweils zwischen den Tags BEGINANTWORT und
  Basistherapie mit Methotrexat. Die Antwortmöglichkeit BEGINANTWORT  Kombinationstherapie mit Schmerzmitteln (NSAR plus Opioid) ist falsch. Die Antwortmöglichkeit BEGINANTWORT  Basistherapie mit Sulfasalazin ist falsch. Die Antwortmöglichkeit BEGINANTWORT  Basistherapie mit Leflunomid ist falsch. Die Antwortmöglichkeit BEGINANTWORT  Therapie mit einem Anti-TNF-Antikörper ist falsch.
1.  A. K. Doherty, J. A. S. Dalbeth: The role of biologics in the management of rheumatoid arthritis. In: Expert opinion on biological therapy. Band 15, Nummer 11, November 2015, S. 1389–1399, doi:10.1517/14712598.2015.1092209, PMID 26452229, PMC 4640293 (freier Volltext).
2.  M. A. Kavanaugh, M. A. Haire: Rheumatoid arthritis. In: The Lancet. Band 389, Nummer 10080, April 2017, S. 1129–1140, doi:10.1016/S0140-6736(16)32465-5, PMID 28255617.
3.  M. A. Kavanaugh, M. A. Haire: Rheumatoid arthritis. In: The Lancet. Band 389, Nummer 10080, April 2017, S. 1129–1140, doi:10.1016
  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwortmöglichkeit ist BEGINANTWORT  Angiotensin-Converting-Enzym-Hemmer. Die korrekte Antwort
 3. Die Antwortmöglichkeit 1 ist falsch, weil die Leitlinien keine Alpha-1-Adrenozeptorblocker mehr empfehlen. Die Antwortmöglichkeit 2 ist falsch, weil die Leitlinien keine Serenoa-repens-Extrakt mehr empfehlen. Die Antwortmöglichkeit 4 ist falsch, weil die Leitlinien keine transurethralen Prostataresektionen mehr empfehlen. Die Antwortmöglichkeit 5 ist falsch, weil die Leitlinien keine Phosphodiesterase-5-Hemmer mehr empfehlen. Die Antwortmöglichkeit 6 ist falsch, weil die Leitlinien keine Kombination aus Alpha-1-Adrenozeptorblocker und Phosphodiesterase-5-Hemmer mehr empfehlen. Die Antwortmöglichkeit
 4. Die korrekte Antwort ist BEGINANTWORT 4. Die Antwortmöglichkeit BEGINANTWORT 4 ist die korrekte Antwort. Die Antwortmöglichkeit BEGINANTWORT 4 ist die korrekte Antwort. Die Antwortmöglichkeit BEGINANTW
 3.

## Einzelnachweise

1.
.
  Hysterektomie mit Adnexektomie beidseits.
Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die Antwortmöglichkeiten werden in der Regel in Form von Text eingegeben. Die
.
Die Patientin ist 77 Jahre alt und hat einen 10 cm großen, hoch aufgebauten, irregulären Endometriumkarzinom. Sie hat keine Vorerkrankungen und keine Operationen vor. Sie hat eine Appendektomie durchgeführt, als junger Erwachsener. Sie hat keine Metastasen. Sie hat eine Laparoskopie mit Hysterektomie und Adnexektomie beidseits durchgeführt. Sie hat keine weitere Maßnahme empfohlen.
Die Patientin ist 77 Jahre alt und hat einen 10 cm großen, hoch aufgebauten, irregulären Endometriumkarzinom. Sie hat keine Vorerkrankungen und keine Operationen vor. Sie hat eine Appendektomie durchgeführt, als junger Erwachsener. Sie hat keine Metastasen. Sie hat eine Laparoskopie mit Hysterektomie und Adnexektomie beidseits durchgeführt. Sie hat keine weitere Maßnahme empfohlen.
Die Patientin ist 77 Jahre alt und hat einen 10 cm großen, hoch aufgebauten, irregulären Endometriumkarzinom. Sie hat keine Vorerkrankungen und keine Operationen vor. Sie hat eine Appendektomie durchgeführt, als junger Erwachsener. Sie hat keine Metastasen. Sie hat eine Laparoskopie mit Hysterektomie und Adnexektomie beidseits durchgeführt. Sie hat keine weitere Maßnahme empfohlen.
Die Patientin ist 77 Jahre alt und hat einen 10 cm großen, hoch aufgebauten, irregulären Endometriumkarzinom. Sie hat keine Vorerkrankungen und keine Operationen vor. Sie hat eine Appendektomie durchgeführt, als junger Erwachsener. Sie hat keine Metastasen. Sie hat eine Laparoskopie mit Hysterektomie und Ad